### Edgar Moises Hernandez-Gonzalez
#### 05/06/20-09/06/20
#### Clasificacion de imagenes motoras en EEG con CNN-2D
##### Caracteristicas = Espectrogramas STFFT (3D)
##### Concatenacion vertical
## Resize proporcional de 264x8
##### 2 seg

In [0]:
from google.colab import drive

In [2]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV
from scipy.signal import spectrogram
import cv2
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Flatten, Dense, Dropout
from keras.optimizers import Adam
from keras.wrappers.scikit_learn import KerasClassifier
from keras import backend as K

Using TensorFlow backend.


In [4]:
# para que este script funcione debe de decir 'channels_last', si dice 'channels_first' no sirve
K.image_data_format()

'channels_last'

In [0]:
x_train = pd.read_csv("/content/drive/My Drive/BCI-IV-2a/RAW/MI-EEG-A06T.csv", header=None)
x_test = pd.read_csv("/content/drive/My Drive/BCI-IV-2a/RAW/MI-EEG-A06E.csv", header=None)
y_train = pd.read_csv("/content/drive/My Drive/BCI-IV-2a/RAW/EtiquetasA06T.csv", header=None)
y_test = pd.read_csv("/content/drive/My Drive/BCI-IV-2a/RAW/EtiquetasA06E.csv", header=None)

In [6]:
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(288, 22000)
(288, 22000)
(288, 1)
(288, 1)


In [0]:
x = pd.concat([x_train, x_test])
y = pd.concat([y_train, y_test])

In [8]:
print(x.shape)
print(y.shape)

(576, 22000)
(576, 1)


In [9]:
# en caso de no tener el y_test se puede utilizar el x_test
n_samples = len(y)

print("n_samples:", n_samples)

n_samples: 576


In [10]:
# esto se podria calcular asi n_clases = len(np.unique(y)), la y puede ser train o test
n_classes = len(np.unique(y))

print("n_classes:", n_classes)

n_classes: 4


In [11]:
# codificar etiquetas con valores de 0 a nClases-1
y = LabelEncoder().fit_transform(y)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:251: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [0]:
#unir 22 canales y resize de tamaño proporcional
def unir_espectrogramas_vertical_resize(data):
  dim = (8, 264) #ancho, alto
  fs = 250
  datos = np.zeros((576,264,8))
  temporal = np.zeros((528,4))
  for i in range(data.shape[0]): #288 muestras
    for j in range(22): #22 canales
      sig = data.iloc[i, j*1000:(j+1)*1000]
      #sig = sig[124:624]
      f, t, Sxx = spectrogram(sig, fs) #espectrograma de STFFT
      temporal[j*24:(j+1)*24, :] = Sxx[8:32, :]
    resized = cv2.resize(temporal, dim, interpolation=cv2.INTER_AREA)
    datos[i] = resized
    if i % 100 == 0: #esto solo es para ver como avanza
      print(i)
  return datos

In [13]:
#llamar a unir_espectrogramas_vertical_resize(data)
inicio = time.time()

x = unir_espectrogramas_vertical_resize(x)

fin = time.time()
print("Tiempo:", fin - inicio)

0
100
200
300
400
500
Tiempo: 7.012707710266113


In [14]:
print(x.shape)

(576, 264, 8)


In [15]:
print(np.max(x))
print(np.min(x))
print(np.mean(x))

185.7043801672993
9.105958279984283e-05
2.179787379488556


In [0]:
#convertir a float
x = x.astype('float32')

#escalar los valores en un rango de 0 a 1 (normalizar)
x /= np.ceil(np.max(x))

In [17]:
print(np.max(x))
print(np.min(x))
print(np.mean(x))

0.99841064
4.8956764e-07
0.011719292


In [18]:
#convertir de 3D a 4D
x = x.reshape((x.shape[0], x.shape[1], x.shape[2], 1))

print(x.shape)

(576, 264, 8, 1)


In [0]:
def CNN_2D(n_filtros, tamano_filtros, n_neuronas):
  model = Sequential()
  model.add(Conv2D(n_filtros, tamano_filtros, activation='relu', padding='same',
                   input_shape=x.shape[1:]))
  model.add(MaxPool2D((2,2)))
  model.add(Conv2D(n_filtros, tamano_filtros, activation='relu', padding='same'))
  model.add(MaxPool2D((2,2)))
  model.add(Flatten())
  model.add(Dense(n_neuronas, activation='relu'))
  model.add(Dropout(0.5))
  model.add(Dense(n_classes, activation='softmax'))

  optimizer = Adam(lr=1e-4)
  model.compile(optimizer = optimizer,
                loss = 'sparse_categorical_crossentropy',
                metrics = ['accuracy'])
  return model

In [0]:
clf = KerasClassifier(build_fn = CNN_2D, epochs=1000, batch_size=36, verbose=0)

In [0]:
n_filtros = [4, 8, 16, 32]
tamano_filtros = [(3,3), (12,2), (12,3)]
n_neuronas = [16, 32, 64, 128]
hiperparametros = dict(n_filtros=n_filtros,
                       tamano_filtros=tamano_filtros,
                       n_neuronas=n_neuronas)

In [22]:
# 48 redes tardan 2474seg
# entonces 32 redes = 1649
4*3*4

48

In [0]:
grid = GridSearchCV(clf, hiperparametros, cv=2)

In [24]:
inicio = time.time()

grid.fit(x, y)

fin = time.time()
print("Tiempo:", fin - inicio)

Tiempo: 3685.4118077754974


In [25]:
print(grid.best_params_)
print(grid.best_score_)

{'n_filtros': 32, 'n_neuronas': 32, 'tamano_filtros': (3, 3)}
0.435763880610466


In [26]:
means = grid.cv_results_['mean_test_score']
stds = grid.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, grid.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r"
        % (mean, std * 2, params))

0.385 (+/-0.042) for {'n_filtros': 4, 'n_neuronas': 16, 'tamano_filtros': (3, 3)}
0.361 (+/-0.042) for {'n_filtros': 4, 'n_neuronas': 16, 'tamano_filtros': (12, 2)}
0.366 (+/-0.003) for {'n_filtros': 4, 'n_neuronas': 16, 'tamano_filtros': (12, 3)}
0.356 (+/-0.031) for {'n_filtros': 4, 'n_neuronas': 32, 'tamano_filtros': (3, 3)}
0.365 (+/-0.042) for {'n_filtros': 4, 'n_neuronas': 32, 'tamano_filtros': (12, 2)}
0.380 (+/-0.087) for {'n_filtros': 4, 'n_neuronas': 32, 'tamano_filtros': (12, 3)}
0.399 (+/-0.007) for {'n_filtros': 4, 'n_neuronas': 64, 'tamano_filtros': (3, 3)}
0.399 (+/-0.007) for {'n_filtros': 4, 'n_neuronas': 64, 'tamano_filtros': (12, 2)}
0.384 (+/-0.010) for {'n_filtros': 4, 'n_neuronas': 64, 'tamano_filtros': (12, 3)}
0.394 (+/-0.010) for {'n_filtros': 4, 'n_neuronas': 128, 'tamano_filtros': (3, 3)}
0.387 (+/-0.045) for {'n_filtros': 4, 'n_neuronas': 128, 'tamano_filtros': (12, 2)}
0.398 (+/-0.003) for {'n_filtros': 4, 'n_neuronas': 128, 'tamano_filtros': (12, 3)}
0.382

In [0]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime → "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Sat Jun  6 22:46:08 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.82       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   76C    P0    31W /  70W |    671MiB / 15079MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  